## Converting Existing LeRobot Datasets from v2.1 to v3.0

Hi! You've stumbled upon a notebook by mandryl.io for converting LeRobot v2.1 datasets to the newest, v3.0 format.

We created this because we still experienced some workflow difficulties when running our own experiments and thought this would be a neat way to help people overcome early setup hassles so that you can get to what matters with your time and ambitions - training AI models for robots!

We kindly ask that for any datasets you port over, please ensure you attribute any of the original dataset sources :)

### Prerequisites

- A personal token from `huggingface` with read/write repo permissions
- A `repo_id` for a dataset you wish to port from `v2.1` to `v3.0`

In [1]:
%pip install lerobot==0.4.3 --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.6/993.6 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 26.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 91.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 2.5 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 12.5 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import huggingface_hub
huggingface_hub.login()

### Configuration

Fill in the two values below and then run the rest of the notebook — no other changes needed.

In [3]:
ORIGINAL_REPO_ID = input("Enter the repo ID of the v2.1 dataset to convert (e.g. youliangtan/so101-table-cleanup): ").strip()
DESTINATION_REPO_ID = input("Enter the repo ID where you want to upload the v3.0 dataset (e.g. my-org/so101-table-cleanup): ").strip()

print(f"\nSource:      {ORIGINAL_REPO_ID}")
print(f"Destination: {DESTINATION_REPO_ID}")


Source:      5hadytru/so101_IF_7
Destination: mandryl-io/so101_IF_7


### Step 1 — Download the v2.1 dataset

In [6]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_dataset_dir = Path("./datasets")
local_dataset_dir.mkdir(exist_ok=True)

local_path = snapshot_download(
    repo_id=ORIGINAL_REPO_ID,
    repo_type="dataset",
    local_dir=local_dataset_dir / ORIGINAL_REPO_ID,
)

print(f"Dataset downloaded to: {local_path}")

Returning existing local_dir `datasets/5hadytru/so101_IF_7` as remote repo cannot be accessed in `snapshot_download` (429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/5hadytru/so101_IF_7/revision/main (Request ID: Root=1-69930469-4a1c80e006c697183ecbe4d9;40535b95-5c12-4464-81b4-4983c087de38)

We had to rate limit your IP (34.158.62.229). To continue using our service, create a HF account or login to your existing account, and make sure you pass a HF_TOKEN if you're using the API.).

We had to rate limit your IP (34.158.62.229). To continue using our service, create a HF account or login to your existing account, and make sure you pass a HF_TOKEN if you're using the API.).


Dataset downloaded to: /content/datasets/5hadytru/so101_IF_7


### Step 2 — Convert to v3.0

In [7]:
from lerobot.datasets.v30.convert_dataset_v21_to_v30 import convert_dataset

convert_dataset(
    repo_id=ORIGINAL_REPO_ID,
    branch=None,
    data_file_size_in_mb=None,
    video_file_size_in_mb=None,
    root=str(local_dataset_dir),
    push_to_hub=False,
    force_conversion=False,
)

print("Conversion complete!")

ValueError: Local dataset has codebase version 'v3.0', expected 'v2.1'. This script is specifically for converting v2.1 datasets to v3.0.

### Step 3 — Upload to Hugging Face

In [ ]:
import subprocess, sys

local_converted_path = str(local_dataset_dir / ORIGINAL_REPO_ID)

subprocess.run(
    [sys.executable, "-m", "huggingface_hub", "repo", "create",
     DESTINATION_REPO_ID, "--type", "dataset"],
    check=False,
)

subprocess.run(
    [sys.executable, "-m", "huggingface_hub", "upload",
     DESTINATION_REPO_ID, local_converted_path, "--repo-type", "dataset"],
    check=True,
)

print(f"Uploaded to {DESTINATION_REPO_ID}")

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.create_tag(
    repo_id=DESTINATION_REPO_ID,
    repo_type="dataset",
    tag="v3.0",
    tag_message="Tagging v3.0 to match codebase_version",
)

print(f"Tagged {DESTINATION_REPO_ID} with v3.0")

### Step 4 — Verify the converted dataset

In [ ]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset

dataset = LeRobotDataset(
    repo_id=DESTINATION_REPO_ID,
    force_cache_sync=True,
)

print(f"Dataset loaded successfully from {DESTINATION_REPO_ID}!")
print(f"Dataset metadata: {dataset.meta}")